# Ingesting Data Into The Cloud

In this section, we will describe a typical scenario in which an application writes data into an Amazon S3 Data Lake and the data needs to be accessed by both the data science / machine learning team, as well as the business intelligence / data analyst team as shown in the figure below.

![](../data/readme_pics/data-ingestion.png)

As a **data scientist or machine learning engineer**, you want to have access to all of the raw data, and be able to quickly explore it. We will show you how to leverage **Amazon Athena** as an interactive query service to analyze data in Amazon S3 using standard SQL, without moving the data. 
* In the first step, we will register the TSV data in our S3 bucket with Athena, and then run some ad-hoc queries on the dataset. 
* We will also show how you can easily convert the TSV data into the more query-optimized, columnar file format Apache Parquet. 

Your **business intelligence team and data analysts** might also want to have a subset of the data in a data warehouse which they can then transform, and query with their standard SQL clients to create reports and visualize trends. We will show you how to leverage **Amazon Redshift**, a fully managed data warehouse service, to 

* insert TSV data into Amazon Redshift, but also be able to combine the data warehouse queries with the data that’s still in our S3 data lake via **Amazon Redshift Spectrum**. 
* You can also use Amazon Redshift’s data lake export functionality to unload data back into our S3 data lake in Parquet file format. 

# Amazon Customer Reviews Dataset

https://s3.amazonaws.com/dsoaws/amazon-reviews-pds/readme.html

### Dataset Columns:

- `marketplace`: 2-letter country code (in this case all "US").
- `customer_id`: Random identifier that can be used to aggregate reviews written by a single author.
- `review_id`: A unique ID for the review.
- `product_id`: The Amazon Standard Identification Number (ASIN).  `http://www.amazon.com/dp/<ASIN>` links to the product's detail page.
- `product_parent`: The parent of that ASIN.  Multiple ASINs (color or format variations of the same product) can roll up into a single parent.
- `product_title`: Title description of the product.
- `product_category`: Broad product category that can be used to group reviews (in this case digital videos).
- `star_rating`: The review's rating (1 to 5 stars).
- `helpful_votes`: Number of helpful votes for the review.
- `total_votes`: Number of total votes the review received.
- `vine`: Was the review written as part of the [Vine](https://www.amazon.com/gp/vine/help) program?
- `verified_purchase`: Was the review from a verified purchase?
- `review_headline`: The title of the review itself.
- `review_body`: The text of the review.
- `review_date`: The date the review was written.

# Setup All Dependencies

In [ ]:
!python --version
!pip list
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null
!pip install --disable-pip-version-check -q awscli==1.18.216 boto3==1.16.56 botocore==1.19.56
!pip install --disable-pip-version-check -q sagemaker==2.29.0
!pip install --disable-pip-version-check -q smdebug==1.0.1
!pip install --disable-pip-version-check -q sagemaker-experiments==0.1.26
!conda install -y pytorch==1.6.0 -c pytorch
!pip install --disable-pip-version-check -q tensorflow==2.3.1
!pip install --disable-pip-version-check -q transformers==3.5.1
!pip install --disable-pip-version-check -q PyAthena==2.1.0
!pip install --disable-pip-version-check -q SQLAlchemy==1.3.22
!pip install --disable-pip-version-check -q psycopg2-binary==2.9.1
!pip install --disable-pip-version-check -q stepfunctions==2.0.0rc1
!conda install -y zip
!pip install --disable-pip-version-check -q matplotlib==3.1.3
!pip install --disable-pip-version-check -q seaborn==0.10.0
setup_dependencies_passed = True
%store setup_dependencies_passed

# Create S3 Bucket

In [ ]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)
print("Default bucket: {}".format(bucket))
setup_s3_bucket_passed = True
%store setup_s3_bucket_passed

# IAM Roles and Policies

In [ ]:
from botocore.config import Config
config = Config(retries={"max_attempts": 10, "mode": "adaptive"})
iam = boto3.client("iam", config=config)
# Get SageMaker Execution Role Name
role_name = role.split("/")[-1]
print("Role name: {}".format(role_name))
setup_iam_roles_passed = False

admin = False

post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]
for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        setup_iam_roles_passed = True
        print("All set")
        break

if not admin:   
        print("*************** [ERROR] *****************")

In [ ]:
if not admin:
    pre_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]

    required_policies = ["IAMFullAccess"]

    for pre_policy in pre_policies:
        for role_req in required_policies:
            if pre_policy["PolicyName"] == role_req:
                print("Attached: {}".format(pre_policy["PolicyName"]))
                try:
                    required_policies.remove(pre_policy["PolicyName"])
                except:
                    pass

    if len(required_policies) > 0:
        print(
            "*************** [ERROR] You need to attach the following policies in order to continue with this workshop *****************\n"
        )
        for required_policy in required_policies:
            print("Not Attached: {}".format(required_policy))
    else:
        print("[OK] You are all set to continue with this notebook!")
else:
    print("[OK] You are all set to continue with this notebook!")

from botocore.exceptions import ClientError

try:
    policy = "AdministratorAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)


try:
    policy = "AmazonSageMakerFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)


try:
    policy = "IAMFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)


try:
    policy = "AmazonS3FullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)


try:
    policy = "ComprehendFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)


try:
    policy = "AmazonAthenaFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)



try:
    policy = "SecretsManagerReadWrite"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)



try:
    policy = "AmazonRedshiftFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)


try:
    policy = "AmazonEC2ContainerRegistryFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)


try:
    policy = "AWSStepFunctionsFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)



try:
    policy = "AmazonKinesisFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)



try:
    policy = "AmazonKinesisFirehoseFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)



try:
    policy = "AmazonKinesisAnalyticsFullAccess"
    response = iam.attach_role_policy(PolicyArn="arn:aws:iam::aws:policy/{}".format(policy), RoleName=role_name)
    print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
except ClientError as e:
    if e.response["Error"]["Code"] == "EntityAlreadyExists":
        print("[OK] Policy is already attached.")
    elif e.response["Error"]["Code"] == "LimitExceeded":
        print("[OK]")
    else:
        print("*************** [ERROR] {} *****************".format(e))

time.sleep(5)




# role = iam.get_role(RoleName=role_name)
post_policies = iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]

required_policies = [
    "AdministratorAccess",
    "SecretsManagerReadWrite",
    "IAMFullAccess",
    "AmazonS3FullAccess",
    "AmazonAthenaFullAccess",
    "ComprehendFullAccess",
    "AmazonEC2ContainerRegistryFullAccess",
    "AmazonRedshiftFullAccess",
    "AWSStepFunctionsFullAccess",
    "AmazonSageMakerFullAccess",
    "AmazonKinesisFullAccess",
    "AmazonKinesisFirehoseFullAccess",
    "AmazonKinesisAnalyticsFullAccess",
]

admin = False

for post_policy in post_policies:
    if post_policy["PolicyName"] == "AdministratorAccess":
        admin = True
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            break
    else:
        try:
            required_policies.remove(post_policy["PolicyName"])
        except:
            pass

if not admin and len(required_policies) > 0:
    print("*************** [ERROR]*****************")
    for required_policy in required_policies:
        print("Not Attached: {}".format(required_policy))
else:
    setup_iam_roles_passed = True
    print("All Set")

# Copy TSV Data To S3

In [ ]:
!aws s3 ls s3://dsoaws/amazon-reviews-pds/tsv/ 
!aws s3 ls s3://dsoaws/amazon-reviews-pds/parquet/

In [ ]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
sm = boto3.Session().client(service_name="sagemaker", region_name=region)

# Set S3 Source Location (Public S3 Bucket)
s3_public_path_tsv = "s3://dsoaws/amazon-reviews-pds/tsv"
%store s3_public_path_tsv
# Set S3 Destination Location (Our Private S3 Bucket)
s3_private_path_tsv = "s3://{}/amazon-reviews-pds/tsv".format(bucket)
print(s3_private_path_tsv)
%store s3_private_path_tsv
#  Copy Data From the Public S3 Bucket to our Private S3 Bucket in this Account
!aws s3 cp --recursive $s3_public_path_tsv/ $s3_private_path_tsv/ --exclude "*" --include "amazon_reviews_us_Digital_Software_v1_00.tsv.gz"
!aws s3 cp --recursive $s3_public_path_tsv/ $s3_private_path_tsv/ --exclude "*" --include "amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
!aws s3 cp --recursive $s3_public_path_tsv/ $s3_private_path_tsv/ --exclude "*" --include "amazon_reviews_us_Gift_Card_v1_00.tsv.gz"

!aws s3 ls $s3_private_path_tsv/

# Create Athena Database Schema

In [ ]:
ingest_create_athena_db_passed = False
%store -r s3_public_path_tsv
%store -r s3_private_path_tsv

from pyathena import connect
import pandas as pd
database_name = "dsoaws"
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)
if database_name in df_show.values:
    ingest_create_athena_db_passed = True
%store ingest_create_athena_db_passed

# Register TSV Data With Athena
This will create an Athena table in the Glue Catalog (Hive Metastore).

In [ ]:
ingest_create_athena_table_tsv_passed = False
%store -r ingest_create_athena_db_passed
table_name_tsv = "amazon_reviews_tsv"
# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         marketplace string,
         customer_id string,
         review_id string,
         product_id string,
         product_parent string,
         product_title string,
         product_category string,
         star_rating int,
         helpful_votes int,
         total_votes int,
         vine string,
         verified_purchase string,
         review_headline string,
         review_body string,
         review_date string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_tsv, s3_private_path_tsv
)

print(statement)
pd.read_sql(statement, conn)
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)
if table_name_tsv in df_show.values:
    ingest_create_athena_table_tsv_passed = True
%store ingest_create_athena_table_tsv_passed


# Convert TSV Data To Parquet with Athena

In [ ]:
ingest_create_athena_table_parquet_passed = False
%store -r ingest_create_athena_table_tsv_passed
table_name_parquet = "amazon_reviews_parquet"
# SQL statement to execute
statement = """CREATE TABLE IF NOT EXISTS {}.{}
WITH (format = 'PARQUET', external_location = '{}', partitioned_by = ARRAY['product_category']) AS
SELECT marketplace,
         customer_id,
         review_id,
         product_id,
         product_parent,
         product_title,
         star_rating,
         helpful_votes,
         total_votes,
         vine,
         verified_purchase,
         review_headline,
         review_body,
         CAST(YEAR(DATE(review_date)) AS INTEGER) AS year,
         DATE(review_date) AS review_date,
         product_category
FROM {}.{}""".format(
    database_name, table_name_parquet, s3_path_parquet, database_name, table_name_tsv
)

print(statement)
pd.read_sql(statement, conn)

# Load partitions by running `MSCK REPAIR TABLE`
statement = "MSCK REPAIR TABLE {}.{}".format(database_name, table_name_parquet)
print(statement)
pd.read_sql(statement, conn)

# Show Partitions
statement = "SHOW PARTITIONS {}.{}".format(database_name, table_name_parquet)
print(statement)
pd.read_sql(statement, conn)
df_partitions.head(5)

statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

if table_name_parquet in df_show.values:
    ingest_create_athena_table_parquet_passed = True

%store ingest_create_athena_table_parquet_passed

# Sample Query
product_category = "Digital_Software"

statement = """SELECT * FROM {}.{}
    WHERE product_category = '{}' LIMIT 100""".format(
    database_name, table_name_parquet, product_category
)

print(statement)
df = pd.read_sql(statement, conn)
df.head(5)

# Create Amazon Redshift Cluster

Amazon Redshift is a fully managed data warehouse which allows you to run complex analytic queries against petabytes of structured data. Your queries are distributed and parallelized across multiple physical resources, and you can easily scale your Amazon Redshift environment up and down depending on your business needs. 

_Note:  This notebook requires that you are running this SageMaker Notebook Instance in a VPC with access to the Redshift cluster.


**Data Lake S3 vs Data Warehouse Redshift**

One of the fundamental differences between data lakes and data warehouses is that while you ingest and store huge amounts of raw, unprocessed data in your data lake, you normally only load some fraction of your recent data into your data warehouse. Depending on your business and analytics use case, this might be data from the past couple of months, a year, or maybe the past 2 years. 


## Setup IAM Access To Read From S3 and Athena

In [ ]:
from botocore.exceptions import ClientError
from botocore.config import Config

config = Config(
   retries = {
      'max_attempts': 10,
      'mode': 'adaptive'
   }
)
iam = boto3.client('iam', config=config)
sts = boto3.client('sts')
redshift = boto3.client('redshift')
sm = boto3.client('sagemaker')
ec2 = boto3.client('ec2')

# Create AssumeRolePolicyDocument
assume_role_policy_doc = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "redshift.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
} 

iam_redshift_role_name = 'DSOAWS_Redshift'
try:
    iam_role_redshift = iam.create_role(
        RoleName=iam_redshift_role_name,
        AssumeRolePolicyDocument=json.dumps(assume_role_policy_doc),
        Description='DSOAWS Redshift Role'
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Role already exists")
    else:
        print("Unexpected error: %s" % e)

# Get the Role ARN
role = iam.get_role(RoleName='DSOAWS_Redshift')
iam_role_redshift_arn = role['Role']['Arn']
print(iam_role_redshift_arn)

account_id = sts.get_caller_identity()['Account']
print(account_id)

## Create Self-Managed Policies

In [ ]:
my_redshift_to_s3 = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "s3:*",
            "Resource": "*"
        }
    ]
}

my_redshift_to_athena = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "athena:*"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "glue:CreateDatabase",
                "glue:DeleteDatabase",
                "glue:GetDatabase",
                "glue:GetDatabases",
                "glue:UpdateDatabase",
                "glue:CreateTable",
                "glue:DeleteTable",
                "glue:BatchDeleteTable",
                "glue:UpdateTable",
                "glue:GetTable",
                "glue:GetTables",
                "glue:BatchCreatePartition",
                "glue:CreatePartition",
                "glue:DeletePartition",
                "glue:BatchDeletePartition",
                "glue:UpdatePartition",
                "glue:GetPartition",
                "glue:GetPartitions",
                "glue:BatchGetPartition"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetBucketLocation",
                "s3:GetObject",
                "s3:ListBucket",
                "s3:ListBucketMultipartUploads",
                "s3:ListMultipartUploadParts",
                "s3:AbortMultipartUpload",
                "s3:CreateBucket",
                "s3:PutObject"
            ],
            "Resource": [
                "arn:aws:s3:::aws-athena-query-results-*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::athena-examples*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListBucket",
                "s3:GetBucketLocation",
                "s3:ListAllMyBuckets"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "sns:ListTopics",
                "sns:GetTopicAttributes"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "cloudwatch:PutMetricAlarm",
                "cloudwatch:DescribeAlarms",
                "cloudwatch:DeleteAlarms"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Effect": "Allow",
            "Action": [
                "lakeformation:GetDataAccess"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}

my_redshift_to_sagemaker = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "sagemaker:*",
            "Resource": "*"
        }
    ]
}

my_redshift_to_sagemaker_passrole = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "iam:PassRole",
            "Resource": f'arn:aws:iam::{account_id}:role/*'
        }
    ]
}

# Create Policy Objects
try:
    policy_redshift_s3 = iam.create_policy(
      PolicyName='DSOAWS_RedshiftPolicyToS3',
      PolicyDocument=json.dumps(my_redshift_to_s3)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

# Get ARN
policy_redshift_s3_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_RedshiftPolicyToS3'
print(policy_redshift_s3_arn)

try:
    policy_redshift_athena = iam.create_policy(
      PolicyName='DSOAWS_RedshiftPolicyToAthena',
      PolicyDocument=json.dumps(my_redshift_to_athena)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

# Get ARN
policy_redshift_athena_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_RedshiftPolicyToAthena'
print(policy_redshift_athena_arn)

try:
    policy_redshift_sagemaker = iam.create_policy(
      PolicyName='DSOAWS_RedshiftPolicyToSageMaker',
      PolicyDocument=json.dumps(my_redshift_to_sagemaker)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)
# Get ARN
policy_redshift_sagemaker_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_RedshiftPolicyToSageMaker'
print(policy_redshift_sagemaker_arn)

try:
    policy_redshift_sagemaker_passrole = iam.create_policy(
      PolicyName='DSOAWS_RedshiftPolicyToSageMakerPassRole',
      PolicyDocument=json.dumps(my_redshift_to_sagemaker_passrole)
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy already exists")
    else:
        print("Unexpected error: %s" % e)

# Get ARN
policy_redshift_sagemaker_passrole_arn = f'arn:aws:iam::{account_id}:policy/DSOAWS_RedshiftPolicyToSageMakerPassRole'
print(policy_redshift_sagemaker_passrole_arn)

# Attach Policies To Role

# Attach DSOAWS_RedshiftPolicyToAthena policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_athena_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)
# Attach DSOAWS_RedshiftPolicyToS3 policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_s3_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)
        
# Attach DSOAWS_RedshiftPolicyToSageMaker policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_sagemaker_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

# Attach DSOAWS_RedshiftPolicyToSageMakerPassRole policy
try:
    response = iam.attach_role_policy(
        PolicyArn=policy_redshift_sagemaker_passrole_arn,
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)

# Update Trust relationshiops to include both Redshift and SageMaker
my_redshift_to_sagemaker_assumerole = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "redshift.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    },
    {
      "Effect": "Allow",
      "Principal": {
        "Service": "sagemaker.amazonaws.com"
      },
      "Action": "sts:AssumeRole"
    }
  ]
}  
try:
    response = iam.update_assume_role_policy(
        PolicyDocument=json.dumps(my_redshift_to_sagemaker_assumerole),
        RoleName=iam_redshift_role_name
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'EntityAlreadyExists':
        print("Policy is already attached. This is ok.")
    else:
        print("Unexpected error: %s" % e)


### Get Security Group ID 

* Make sure the Redshift VPC is the same this notebook is running within
* Make sure the VPC has the following 2 properties enabled
 *     DNS resolution = Enabled
 *     DNS hostnames = Enabled
* This allows private, internal access to Redshift from this SageMaker notebook using the fully qualified endpoint name.

In [ ]:
try:
    domain_id = sm.list_domains()['Domains'][0]['DomainId']
    describe_domain_response = sm.describe_domain(DomainId=domain_id)
    vpc_id = describe_domain_response['VpcId']
    security_groups = ec2.describe_security_groups()['SecurityGroups']
    for security_group in security_groups:
        if vpc_id == security_group['VpcId']:
            security_group_id = security_group['GroupId']
    print(security_group_id)    
except:
    pass

try:
    notebook_instance_name = sm.list_notebook_instances()['NotebookInstances'][0]['NotebookInstanceName']
    notebook_instance = sm.describe_notebook_instance(NotebookInstanceName=notebook_instance_name)
    security_group_id = notebook_instance['SecurityGroups'][0]
    print(security_group_id)    
except:
    pass

### Create Secret in Secrets Manager

AWS Secrets Manager is a service that enables you to easily rotate, manage, and retrieve database credentials, API keys, and other secrets throughout their lifecycle. Using Secrets Manager, you can secure and manage secrets used to access resources in the AWS Cloud, on third-party services, and on-premises.

In [ ]:
secretsmanager = boto3.client('secretsmanager')

try:
    response = secretsmanager.create_secret(
        Name='dsoaws_redshift_login',
        Description='DSOAWS Redshift Login',
        SecretString='[{"username":"dsoaws"},{"password":"Password9"}]',
        Tags=[
            {
                'Key': 'name',
                'Value': 'dsoaws_redshift_login'
            },
        ]
    )
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceExistsException':
        print("Secret already exists. This is ok.")
    else:
        print("Unexpected error: %s" % e)

In [ ]:
import json

secret = secretsmanager.get_secret_value(SecretId='dsoaws_redshift_login')
cred = json.loads(secret['SecretString'])

master_user_name = cred[0]['username']
master_user_pw = cred[1]['password']

# Redshift configuration parameters
redshift_cluster_identifier = 'dsoaws'
database_name = 'dsoaws'
cluster_type = 'multi-node'

# Note that only some Instance Types support Redshift Query Editor 
# (https://docs.aws.amazon.com/redshift/latest/mgmt/query-editor.html)
node_type = 'dc2.large'
number_nodes = '2' 

response = redshift.create_cluster(
        DBName=database_name,
        ClusterIdentifier=redshift_cluster_identifier,
        ClusterType=cluster_type,
        NodeType=node_type,
        NumberOfNodes=int(number_nodes),       
        MasterUsername=master_user_name,
        MasterUserPassword=master_user_pw,
        IamRoles=[iam_role_redshift_arn],
        VpcSecurityGroupIds=[security_group_id],
        Port=5439,
        PubliclyAccessible=False
)

print(response)

import time

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

# Load TSV Data From S3/Athena into Redshift

We can leverage our previously created table in Amazon Athena with its metadata and schema information stored in the AWS Glue Data Catalog to access our data in S3 through Redshift Spectrum. All we need to do is create an external schema in Redshift, point it to our AWS Glue Data Catalog, and point Redshift to the database we’ve created.  


In [ ]:
secret = secretsmanager.get_secret_value(SecretId='dsoaws_redshift_login')
cred = json.loads(secret['SecretString'])

master_user_name = cred[0]['username']
master_user_pw = cred[1]['password']
redshift_cluster_identifier = 'dsoaws'

database_name_redshift = 'dsoaws'
database_name_athena = 'dsoaws'

redshift_port = '5439'

schema_redshift = 'redshift'
schema_athena = 'athena'

table_name_tsv = 'amazon_reviews_tsv'
import time

response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
cluster_status = response['Clusters'][0]['ClusterStatus']
print(cluster_status)

while cluster_status != 'available':
    time.sleep(10)
    response = redshift.describe_clusters(ClusterIdentifier=redshift_cluster_identifier)
    cluster_status = response['Clusters'][0]['ClusterStatus']
    print(cluster_status)

# Get Redshift Endpoint Address & IAM Role
redshift_endpoint_address = response['Clusters'][0]['Endpoint']['Address']
iam_role = response['Clusters'][0]['IamRoles'][0]['IamRoleArn']

print('Redshift endpoint: {}'.format(redshift_endpoint_address))
print('IAM Role: {}'.format(iam_role))

# Create Redshift Connection
import awswrangler as wr

con_redshift = wr.data_api.redshift.connect(
    cluster_id=redshift_cluster_identifier,
    database=database_name_redshift,
    db_user=master_user_name,
)

# Redshift Spectrum
Amazon Redshift Spectrum directly queries data in S3, using the same SQL syntax of Amazon Redshift. You can also run queries that span both the frequently accessed data stored locally in Amazon Redshift and your full datasets stored cost-effectively in S3.

To use Redshift Spectrum, your cluster needs authorization to access data catalog in Amazon Athena and your data files in Amazon S3. You provide that authorization by referencing an AWS Identity and Access Management (IAM) role that is attached to your cluster. 

To use this capability in from your Amazon SageMaker notebook:

* Register your Athena database `dsoaws` with Redshift Spectrum
* Query Your Data in Amazon S3



In [ ]:
statement = """
CREATE EXTERNAL SCHEMA IF NOT EXISTS {} FROM DATA CATALOG 
    DATABASE '{}' 
    IAM_ROLE '{}'
    REGION '{}'
    CREATE EXTERNAL DATABASE IF NOT EXISTS
""".format(schema_athena, database_name_athena, iam_role, region_name)

print(statement)
wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)
statement = """
SELECT product_category, COUNT(star_rating) AS count_star_rating
    FROM {}.{}
    GROUP BY product_category
    ORDER BY count_star_rating DESC
""".format(schema_athena, table_name_tsv)

print(statement)

df = wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

df.head()

# Load TSV Data Into Redshift

Create Redshift tables with Customer Reviews data for each year we wish to load.

In [ ]:
# Create `redshift` Schema
statement = """CREATE SCHEMA IF NOT EXISTS {}""".format(schema_redshift)

wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)


# Create Redshift Tables for Each Year We Wish to Load
When you create a table, you can specify one or more columns as the **sort key**. Amazon Redshift stores your data on disk in sorted order according to the sort key. This means, you can optimize your table by choosing a sort key that reflects your most frequently used query types. If you query a lot of recent data, you can specify a timestamp column as the sort key. If you frequently query based on range or equality filtering on one column, you should choose that column as the sort key. 

As we are going to run a lot of queries in the next chapter filtering on `product_category`, let’s choose that one as our sort key. 

You can also define a distribution style for every table. When you load data into a table, Redshift distributes the rows of the table among your cluster nodes according to the table’s distribution style. When you run a query, the query optimizer redistributes the rows to the cluster nodes as needed to perform any joins and aggregations. So our goal should be to optimize the rows distribution to minimize needed data movements. There are three distribution styles from which you can choose from: 

* KEY distribution - distribute the rows according to the values in one column
* ALL distribution - distribute a copy of the entire table to every node
* EVEN distribution - the rows are distributed across all nodes in a round-robin-fashion which is the default distribution style

For our table, we’ve chosen **KEY distribution** based on `product_id` as this column has a high cardinality, shows an even distribution and can be used to join with other tables. 

Now we are ready to copy the data from S3 into our new Redshift table. 


In [ ]:
# Create table function, pass session, table name prefix and start & end year

def create_redshift_table_tsv(wr, con_redshift, table_name_prefix, start_year, end_year):
    for year in range(start_year, end_year + 1, 1):
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
        CREATE TABLE IF NOT EXISTS redshift.{}( 
             marketplace varchar(2),
             customer_id varchar(8),
             review_id varchar(14),
             product_id varchar(10) DISTKEY,
             product_parent varchar(9),
             product_title varchar(400),
             product_category varchar(24),
             star_rating int,
             helpful_votes int,
             total_votes int,
             vine varchar(1),
             verified_purchase varchar(1),
             review_headline varchar(128),
             review_body varchar(65535),
             review_date varchar(10),
             year int)  SORTKEY (product_category)
        """.format(current_table_name)

        wr.data_api.redshift.read_sql_query(
            sql=statement,
            con=con_redshift,
        )
    print("Done.")
create_redshift_table_tsv(wr, con_redshift, 'amazon_reviews_tsv', 2014, 2015)

# Insert TSV Data into New Redshift Tables

For such bulk inserts, you can either use a `COPY` command, or an `INSERT INTO` command. In general, the `COPY` command is preferred, as it loads data in parallel and more efficiently from Amazon S3, or other supported data sources. 

If you are loading data or a subset of data from one table into another, you can use the `INSERT INTO` command with a `SELECT` clause for high-performance data insertion. As we’re loading our data from the `athena.amazon_reviews_tsv` table, let’s choose this option. 


In [ ]:
# INSERT INTO function, pass session, table name prefix and start & end year

def insert_into_redshift_table_tsv(wr, con_redshift, table_name_prefix, start_year, end_year):
    for year in range(start_year, end_year + 1, 1):
        print(year)
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
            INSERT 
            INTO
                redshift.{}
                SELECT
                    marketplace,
                    customer_id,
                    review_id,
                    product_id,
                    product_parent,
                    product_title,
                    product_category,
                    star_rating,
                    helpful_votes,
                    total_votes,
                    vine,
                    verified_purchase,
                    review_headline,
                    review_body,
                    review_date,
                    CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year
                FROM
                    athena.amazon_reviews_tsv             
                WHERE
                    year = {}
            """.format(current_table_name, year)

        wr.data_api.redshift.read_sql_query(
            sql=statement,
            con=con_redshift,
        )

        df.head()
    print("Done.")
    insert_into_redshift_table_tsv(wr, con_redshift, 'amazon_reviews_tsv', 2014, 2015)

# Query Both Athena And Redshift With `Redshift Spectrum`


In [ ]:
# Now Query Across Both Redshift and Athena in a single query
# Use `UNION ALL` across 2 Redshift tables (2015, 2014) and the rest from Athena/S3 (2013-1995)

statement = """
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2015
  GROUP BY redshift.amazon_reviews_tsv_2015.product_category, year
UNION ALL
SELECT year, product_category, COUNT(star_rating) AS count_star_rating
  FROM redshift.amazon_reviews_tsv_2014
  GROUP BY redshift.amazon_reviews_tsv_2014.product_category, year
UNION ALL
SELECT CAST(DATE_PART_YEAR(TO_DATE(review_date, 'YYYY-MM-DD')) AS INTEGER) AS year, product_category, COUNT(star_rating) AS count_star_rating
  FROM athena.amazon_reviews_tsv
  WHERE year <= 2013
  GROUP BY athena.amazon_reviews_tsv.product_category, year
ORDER BY product_category ASC, year DESC
"""

print(statement)
df = wr.data_api.redshift.read_sql_query(
    sql=statement,
    con=con_redshift,
)

df.head()

# When to use Athena vs. Redshift?

### Amazon Athena
Athena should be your preferred choice when running ad-hoc SQL queries on data that is stored in Amazon S3. It doesn’t require you to set up or manage any infrastructure resources, and you don’t need to move any data. It supports structured, unstructured, and semi-structured data. With Athena, you are defining a **“schema on read”** - you basically just log in, create a table and you are good to go. 

### Amazon Redshift
Redshift is targeted for modern data analytics on large sets of structured data. Here, you need to have a predefined **“schema on write”**. Unlike serverless Athena, Redshift requires you to create a cluster (compute and storage resources), ingest the data and build tables before you can start to query, but caters to performance and scale. So for any highly-relational data with a transactional nature (data gets updated), workloads which involve complex joins, and latency requirements to be sub-second, Redshift is the right choice.


# Using Redshift Data Lake Export

Redshift Data Lake Export gives you the ability to unload the result of a Redshift query to your S3 data lake in Apache Parquet format. This enables you to save data transformation and enrichment you have done in Redshift into your S3 data lake in an open format.

You can specify one or more partition columns so that unloaded data is automatically partitioned into folders in your Amazon S3 bucket. 

For example, you can choose to unload our customer reviews data and partition it by `product_category`. This enables your queries to take advantage of partition pruning and skip scanning non-relevant partitions, improving query performance and minimizing cost.

In [ ]:
def unload_redshift_table(wr, con_redshift, table_name_prefix, start_year, end_year, s3_path, iam_role):
    for year in range(start_year, end_year+1, 1):
        current_table_name = table_name_prefix+'_'+str(year)
        statement = """
            UNLOAD ('SELECT marketplace, customer_id, review_id, product_id, product_parent, 
                product_title, product_category, star_rating, helpful_votes, total_votes, 
                vine, verified_purchase, review_headline, review_body, review_date, year 
            FROM redshift.{}')
            TO '{}/{}/'
            IAM_ROLE '{}'
            PARQUET PARALLEL ON 
            PARTITION BY (product_category)
        """.format(current_table_name, s3_path, year, iam_role)

        wr.data_api.redshift.read_sql_query(
            sql=statement,
            con=con_redshift,
        )
    print("Done.")

unload_redshift_table(wr, con_redshift, 'amazon_reviews_tsv', 2014, 2015, s3_path_parquet_unload, iam_role)
print(s3_path_parquet_unload)
!aws s3 ls $s3_path_parquet_unload/2014/
!aws s3 ls $s3_path_parquet_unload/2015/